Import Library

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error


2025-05-06 11:16:19.125477: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-06 11:16:19.134375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746504979.144135  940449 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746504979.146993  940449 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746504979.154659  940449 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Prepare Sequence

In [ ]:
def prepare_sequences(data, window_size):
    """ Prepare sequences for BiLSTM training """
    X, y = [], []
    for i in range(len(data) - window_size):
        seq_x = data[i:(i + window_size)]
        seq_y = data[i + window_size]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y).reshape(-1, 1)

Buidl BiLSTM

In [ ]:

def predict_bilstm(data, window_size):
    # Scale 'casos' data for LSTM using MinMaxScaler
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data['PM10 (ug/m3)'].values.reshape(-1,1))

    # Prepare data for LSTM
    X, y = prepare_sequences(scaled_data, window_size)

    # Define LSTM model
    model = Sequential([
        Bidirectional(LSTM(1000, activation='tanh'), input_shape=(window_size, 1)),
        Dropout(0.3),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Fit the LSTM model
    model.fit(X, y, epochs=150, batch_size=16, verbose=0)

    # Initialize the results DataFrame
    prediction_table = pd.DataFrame()

    # Start the moving-window prediction
    for i in range(len(data) - window_size):  # ensures space for predictions
        # Prepare input sequence for prediction
        input_seq = scaled_data[i:i + window_size]
        input_seq = input_seq.reshape((1, window_size, 1))

        # Predict the next 12 weeks
        predictions = []
        for weeks_ahead in [1, 2, 3, 4, 8, 12]:
            future_index = i + window_size + weeks_ahead - 1
            if future_index < len(data):
                prediction = model.predict(input_seq, verbose=0)[0][0]
                prediction = scaler.inverse_transform([[prediction]])[0][0]
                predictions.append(prediction)

        new_row = {
            'Datetime': data.iloc[i + window_size]['datetime'],
            'PM10 (ug/m3)': data.iloc[i + window_size]['PM10 (ug/m3)'],
            **{f'Predicted_Cases_Week{w}': p for w, p in zip([1, 2, 3, 4, 8, 12], predictions)}
        }
        prediction_table = pd.concat([prediction_table, pd.DataFrame([new_row])], ignore_index=True)

    return prediction_table, model, scaler


Train Model

In [ ]:
data = pd.read_csv('Data/Weekly PM10 in Andhra Pradesh.csv')
data['datetime'] = pd.to_datetime(data['From Date'])
window_size = 14  # Fixed window size
results_df, model, scaler = predict_bilstm(data, window_size)
print(results_df.head())

def lag_predictions(df):
    # Shift the prediction columns to introduce lags
    df['Predicted_Cases_Week2'] = df['Predicted_Cases_Week2'].shift(1)  # Lag by 1
    df['Predicted_Cases_Week3'] = df['Predicted_Cases_Week3'].shift(2)  # Lag by 2
    df['Predicted_Cases_Week4'] = df['Predicted_Cases_Week4'].shift(3)  # Lag by 3
    df['Predicted_Cases_Week8'] = df['Predicted_Cases_Week8'].shift(7)  # Lag by 7
    df['Predicted_Cases_Week12'] = df['Predicted_Cases_Week12'].shift(11) # Lag by 11

    return df

# Use the function
results_df = lag_predictions(results_df)
print(results_df.head(20))  # Print the first 20 rows to see both original and lagged values

results_df.to_csv('Output/BiLSTM_1000.csv', index=False)  # index=False means do not write row names (index)

95% uncertainty interval

In [ ]:
# Detect computation device (GPU if available, otherwise CPU)
device = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"
print(f"Using device: {device}")

# Function to predict with uncertainty using MC Dropout (batch-based)
@tf.function
def predict_with_uncertainty_batch(model, input_seq, n_iter=500):
    input_seq = tf.cast(input_seq, tf.float32)
    input_batch = tf.repeat(input_seq, repeats=n_iter, axis=0)
    preds = model(input_batch, training=True)
    return tf.reduce_mean(preds, axis=0)

# Main function for Adaptive Conformal Prediction (ACP)
def adaptive_conformal_sliding(data, model, scaler, window_size, horizons=[1, 2, 3, 4, 8, 12],
                               W=50, alpha=0.01, n_iter=500):
    scaled_data = scaler.transform(data['PM10 (ug/m3)'].values.reshape(-1, 1))
    results = []

    for h in horizons:
        covered = 0
        total = 0

        for t in range(window_size + h - 1 + W, len(data) - h):
            # Calculate nonconformity scores in the previous window
            window_scores = []
            for w in range(t - W, t):
                seq = scaled_data[w - window_size:w].reshape((1, window_size, 1))
                mean_pred = predict_with_uncertainty_batch(model, seq, n_iter)
                mean_pred = scaler.inverse_transform(np.array(mean_pred).reshape(-1, 1))[0][0]
                true_val = data.iloc[w + h]['PM10 (ug/m3)']
                score = abs(true_val - mean_pred)
                window_scores.append(score)

            # Calculate the (1 - alpha) quantile of the nonconformity scores
            q_t = np.quantile(window_scores, 1 - alpha)

            # Predict at time t using the current model
            seq = scaled_data[t - window_size:t].reshape((1, window_size, 1))
            mean_pred = predict_with_uncertainty_batch(model, seq, n_iter)
            mean_pred = scaler.inverse_transform(np.array(mean_pred).reshape(-1, 1))[0][0]
            lower = mean_pred - q_t
            upper = mean_pred + q_t

            true_val = data.iloc[t + h]['PM10 (ug/m3)']
            if lower <= true_val <= upper:
                covered += 1
            total += 1

        coverage = covered / total * 100 if total > 0 else 0
        results.append({
            'Week': h,
            'Coverage_%': round(coverage, 2),
            'Total': total,
            'Covered': covered
        })

    return pd.DataFrame(results)


Using device: /GPU:0


Check Uncertainty Intervals

In [ ]:
acp_df = adaptive_conformal_sliding(data, model, scaler, window_size, W=50, alpha=0.1, n_iter=500)
print(acp_df)
acp_df.to_csv("Output/ACP_Coverage_Results_BiLSTMusingGraph.csv", index=False)


   Week  Coverage_%  Total  Covered
0     1       86.71    286      248
1     2       89.79    284      255
2     3       88.65    282      250
3     4       87.14    280      244
4     8       88.60    272      241
5    12       87.12    264      230


Evaluate the model

In [ ]:
def calculate_errors(df):
    errors = {}

    for week in [1, 2, 3, 4, 8, 12]:
        actual = df['PM10 (ug/m3)']
        predicted = df[f'Predicted_pollution_Week{week}']

        # Hilangkan NaN sebelum perhitungan error
        valid_idx = ~actual.isna() & ~predicted.isna()
        actual_valid = actual[valid_idx]
        predicted_valid = predicted[valid_idx]

        # Pastikan setidaknya ada satu data valid sebelum menghitung error
        if len(actual_valid) > 0:
            mae = mean_absolute_error(actual_valid, predicted_valid)
            mse = mean_squared_error(actual_valid, predicted_valid)
            rmse = mse ** 0.5
            mape = (abs((actual_valid - predicted_valid) / actual_valid).mean()) * 100
        else:
            mae, mse, rmse, mape = None, None, None, None  # Jika tidak ada data valid, beri nilai None

        errors[f'Week{week}'] = {'MAE': mae, 'MAPE': mape, 'RMSE': rmse, 'MSE': mse}


    return errors

# Hitung error
errors = calculate_errors(results_df)


errors_df = pd.DataFrame(errors).T


In [13]:
errors_df

,MAE,MAPE,RMSE,MSE
Week1,346.633486,4.745286,499.229436,2.492300e+05
Week2,2600.818296,43.838842,3460.205823,1.197302e+07
Week3,3015.172559,55.513506,4086.557726,1.669995e+07
Week4,3342.445354,63.321174,4498.051034,2.023246e+07
Week8,3532.235120,56.010867,4593.070258,2.109629e+07
Week12,3962.006605,67.016100,5033.233789,2.533344e+07


In [ ]:
errors_df.to_csv("Output/BiLSTM1000_150_03.csv") #layer_epoch_dropout